In [ ]:
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import ambient
from ambient import material_data as mats
from ambient.material import MaterialResistanceOnly
from ambient.construction import ConstructionLayered

In [ ]:
con = ConstructionLayered(
    materials=[
        MaterialResistanceOnly(0.060),
        mats.Brickwork(105 / 1000),
        MaterialResistanceOnly(0.180),
        mats.HeavyweightConcrete(100 / 1000),
        MaterialResistanceOnly(0.120),
    ],
    timestep=3600,
)   

In [ ]:
def iterate_heat_transfer(con, temp_in, temp_out, qi, qo=None, iterations=100):
    """Iterate the heat transfer calculation."""
    for iteration in range(iterations):
        for hour in range(24):
            qi[hour] = con.calculate_heat_flux_inside(temp_out, temp_in, qi, hour)
            if qo is not None:
                qo[hour] = con.calculate_heat_flux_outside(temp_out, temp_in, qo, hour)

In [ ]:
SOL_AIR = 273.15 + np.array(
    (   
        24.4, 24.4, 23.8, 23.3, 23.3, 23.8,
        25.5, 27.2, 29.4, 31.6, 33.8, 36.1,
        43.3, 49.4, 53.8, 55.0, 52.7, 45.5,
        30.5, 29.4, 28.3, 27.2, 26.1, 25.0,
    )   
)

qe = np.zeros(24)
qo = np.zeros(24)
temp_in = np.full_like(SOL_AIR, 273.15 + 24)
temp_out = SOL_AIR

# do multiple iterations
iterate_heat_transfer(con, temp_in, temp_out, qe, qo, iterations=100)

In [ ]:
times = np.arange(24)

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces

fig.add_trace(
    go.Bar(x=times, y=qo, name="Heat transfer to outside"),
    #secondary_y=False,
)
fig.add_trace(
    go.Bar(x=times, y=qe, name="Heat transfer to inside"),
    #secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=times, y=SOL_AIR - 273.15, name="Sol-air temp."),
    #secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=times, y=np.full_like(SOL_AIR, 24), name="Inside temp."),
    #secondary_y=True,
)
fig.add_trace(
    go.Bar(x=times, y=con.thermal_transmittance * (SOL_AIR - 273.15 - 24), name="SS heat transfer"),
    #secondary_y=True,
)


# Add figure title
fig.update_layout(
    title_text="Heat transfer rate"
)

# Set x-axis title
fig.update_xaxes(title_text="Hour")

# Set y-axes titles
#fig.update_yaxes(title_text="Temperature (C)", secondary_y=True)
fig.update_yaxes(title_text="Heat transfer rate (W/m2)", secondary_y=False)

fig.show()